In [ ]:
import os
import sys
import re
import random
import pathlib
import findspark

from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.streaming import StreamingContext
from pyspark import SparkConf


os.environ['SPARK_HOME'] = '/Users/audioworkstation/Documents/WORKSPACE/LEARNING/spark_streaming_using_x/spark-3.5.0-bin-hadoop3'
os.environ['PYSPARK_DEIVER_PYTHON'] = 'jupyter'
os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] = 'lab'
os.environ['PYSPARK_PYTHON'] = 'python'


findspark.init()
findspark.find()

random.seed(15)
conf = (SparkConf().setMaster('local[2]').setAppName('TextUpdater').set('spark.executer.memory', '2g'))
sc = SparkContext(conf=conf)
ssc = StreamingContext(sparkContext=sc, batchDuration=10)
ssc.checkpoint('checkpoint')

In [ ]:
lines = ssc.socketTextStream("localhost", 5555)

# Transformations on the DStream
words_dstream = lines.flatMap(lambda line: line.split(" "))
word_pairs_dstream = words_dstream.map(lambda word: (word, 1))

# calculate number of received words in last 30 seconds, every 10 seconds
word_pairs_dstream.countByWindow(windowDuration=30, slideDuration=10).pprint() 

ssc.start()
ssc.awaitTermination()